In [38]:
cd ../..

c:\Users\Josu\Documents\Workspace\Human-Benchmark\src\server\model


In [ ]:
! del /q models\logs\*
! for /d %x in (models\logs\*) do @rd /s /q "%x"

In [39]:
%load_ext tensorboard

In [40]:
import glob
from datetime import datetime

import chess
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
from scripts.utils import absolute_bitboard
from tensorflow.keras import callbacks, utils

In [48]:
XX, yy = [], []
for file in glob.glob("data/test/*.npz"):
    with np.load(file) as f:
        XX.append(f["X"]), yy.append(f["y"])
X, y = np.concatenate(XX), np.concatenate(yy)

In [ ]:
_, x, _ = plt.hist(y, bins=75, density=True)  
ax = plt.plot(x,  stats.gaussian_kde(y)(x))

In [ ]:
model = DeepChess()

In [ ]:
tensor_board = callbacks.TensorBoard(
    log_dir=f"models/logs/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
)
# model_checkpoint = callbacks.ModelCheckpoint(
#     f"models/ser/{datetime.now().strftime('%Y%m%d-%H%M%S')}"
# )

In [49]:
model.compile(
    optimizer="adam",
    loss="mse"
)

In [50]:
_ = model.fit(
    x=X, y=y,
    batch_size=256,
    epochs=5,
    # callbacks=(tensor_board, model_checkpoint),
    validation_split=0.2
)

Epoch 1/5
1260/1260 [==============================] - 34s 27ms/step - loss: 0.1711 - val_loss: 0.1787
Epoch 2/5
1260/1260 [==============================] - 32s 26ms/step - loss: 0.1436 - val_loss: 0.1614
Epoch 3/5
1260/1260 [==============================] - 32s 25ms/step - loss: 0.1345 - val_loss: 0.1567
Epoch 4/5
1260/1260 [==============================] - 34s 27ms/step - loss: 0.1295 - val_loss: 0.1581
Epoch 5/5
1260/1260 [==============================] - 34s 27ms/step - loss: 0.1243 - val_loss: 0.1432


In [65]:
model.save("models/ser/v1")

INFO:tensorflow:Assets written to: models/ser/v1\assets


In [ ]:
# %tensorboard --logdir=models/logs

In [52]:
import chess
from chess import engine
simple_engine = engine.SimpleEngine.popen_uci("lib/stockfish/stockfish.exe")

In [61]:
simple_engine.analyse(
    chess.Board("r1bqkbnr/2pp1ppp/p1n5/1p2p3/B3P3/5N2/PPPP1PPP/RNBQK2R b KQkq - 0 1"),
    engine.Limit(time=.1)
)

{'depth': 11,
 'seldepth': 17,
 'multipv': 1,
 'score': PovScore(Cp(+375), BLACK),
 'nodes': 53844,
 'nps': 527882,
 'tbhits': 0,
 'time': 0.102,
 'pv': [Move.from_uci('b5a4'), Move.from_uci('e1g1')]}

In [63]:
model.predict(
    np.array([
        absolute_bitboard(
            chess.Board("r1bqkbnr/2pp1ppp/p1n5/4p3/p3P3/5N2/PPPP1PPP/RNBQK2R w KQkq - 0 1")
        )
    ])
)

array([[0.11005957]], dtype=float32)

In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras.layers import (Activation, Add, AveragePooling2D, Conv2D, Dense,
                                     Dropout, Flatten, GlobalAveragePooling2D, Input,
                                     Lambda, Multiply, ReLU, Reshape)

In [44]:
model = keras.Sequential(
    layers=(
    	Conv2D(32, (3, 3), activation="relu", input_shape=(8, 8, 7)),
    	Conv2D(64, (3, 3), activation="relu"),
    	GlobalAveragePooling2D(),
    	Dense(32, activation="relu"),
    	Dense(1, activation="tanh")
    )
)
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_60 (Conv2D)           (None, 6, 6, 32)          2048      
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 4, 4, 64)          18496     
_________________________________________________________________
global_average_pooling2d_44  (None, 64)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_61 (Dense)             (None, 1)                 33        
Total params: 22,657
Trainable params: 22,657
Non-trainable params: 0
_________________________________________________________________


In [ ]:
inputs = Input(shape=(8, 8, 7))
FILTERS = 128
y_1 = Conv2D(FILTERS, (3, 3), padding="same", activation="relu")(inputs)
BLOCKS = 10
for _ in range(BLOCKS):
    x = Conv2D(FILTERS, (3, 3), padding="same", activation="relu")(y_1)
    y_2 = Conv2D(FILTERS, (3, 3), padding="same", activation="relu")(x)
    x = GlobalAveragePooling2D()(y_2)
    SE_CHANNELS = 32
    x = Dense(SE_CHANNELS, activation="relu")(x)
    x = Dropout(.2)(x)
    x = Dense(2 * FILTERS)(x)
    x = Dropout(.2)(x)
    x = Reshape((2, FILTERS))(x)
    W, B = Lambda(tf.unstack, arguments=dict(axis=1))(x)
    # THIS IS FIXED, CHANGE IN ORIGINAL
    z = Activation("sigmoid")(W)
    z = Multiply()((z, y_2))
    x = Add()((z, B))
    x = Add()((x, y_1))
    y_1 = ReLU()(x)
VALUE_CONV_SIZE = 32
x = Conv2D(VALUE_CONV_SIZE, (3, 3), padding="same", activation="relu")(y_1)
x = Conv2D(128, (3, 3), activation="relu")(x)
x = Flatten()(x)
outputs = Dense(1, activation="tanh")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()